In [1]:
WAITING = 0
BOARDED = 1
COST_PER_TIMESTEP = 0.2
REJECT_BUSSTOP_COST = 10
CROWDED_COST_MULTIPLIER = 0.05
import random
import numpy

In [2]:
class People:

    def __init__(self):
        self.timeWaited = 0
        self.status = WAITING
    
    def chooseDisembarkLocation(curStop,route):
        # fetch route details
        routeDetails = Route.getDetails(route)

        # function to check distance in number of bus stops from start
        # we call this distance removal
        def fn(reqd_stop,deets):
            deets[reqd_stop]['removal'] = 0
            x = 0

            # forward pass - add distance in bus stops in direction of the loop
            stop = deets[reqd_stop]['nextStop']
            deets[stop]['prev'] = reqd_stop
            while stop!= reqd_stop:
                x+=1
                deets[stop]['removal'] = x
                nextStop = deets[stop]['nextStop']
                deets[nextStop]['prev'] = stop
                stop = nextStop

            # backward pass - add distance in bus stops opposite direction of the loop
            prev = deets[stop]['prev']
            x= 1
            deets[prev]['removal'] = min(deets[prev]['removal'],x)
            while prev!= reqd_stop:
                prev = deets[prev]['prev']
                x+=1
                deets[prev]['removal'] = min(deets[prev]['removal'],x)
            return deets

        # take total of removals
        routeDetails = fn(curStop,routeDetails)
        total = 0
        for key,value in routeDetails.items():
            if key==curStop:
                continue
            total+=value['removal']

        # generate random value from 0 to total - 1
        randVal = random.randint(0,total-1)
        
        # pick the cumulative region in which the random value falls and return that region
        total = 0
        for key,value in routeDetails.items():
            if key==curStop:
                continue
            total+=value['removal']
            if randVal<total:
                break
        
        return key


    def board(self):
        self.status = BOARDED

    def step(self):
        if self.status == WAITING:
            self.timeWaited += 1
            return COST_PER_TIMESTEP
        else:
            return 0

In [15]:
class Bus:

    # replace start location with starting location in the format: (lastVistedStop,distance)
    def __init__(self, route, startLocation = (" SBS ",1)):

        # graph in the format {"currentBS":{"nextStop":"nextBS","roadWeight":distance}}
        if route == "blue":
            startLocation = (" NIE ",1)
        # constants
        self.route = route #red/blue
        self.startLocation = startLocation
        self.maxCapacity = 60
        self.seats = 30
        self.discomfortCapacity = 45

        # variables        
        self.driverRounds = 0
        self.currentLocation = startLocation
        self.currentCapacity = 0
        self.peopleDisembarkDict = {}

        # initialising peopleDisembarkDict
        routeDetails = Route.getDetails(self.route)
        for key in list(routeDetails.keys()):
            self.peopleDisembarkDict[key] = []

    def __str__(self):
        return str(self.currentCapacity) + '   ' + str(self.currentLocation)
            
    def step(self,main):
        cost = 0
        # store last visited stop and distance from last visited stop
        lastVistedStop = self.currentLocation[0]
        distance = self.currentLocation[1]

        # fetch route deets
        routeDetails = Route.getDetails(self.route)

        # check whether next stop has been reached
        if routeDetails[lastVistedStop]["roadWeight"] == distance+1:

            # store nextStop and update current location
            nextStop = routeDetails[lastVistedStop]["nextStop"]

            self.currentLocation = (nextStop,0)

            # visit the next bus stop
            self.visitBusStop(nextStop,main)

        else:

            # update current location
            self.currentLocation = (lastVistedStop, distance+1)
        
        if self.currentLocation == self.startLocation:
            self.driverRounds+=1

        cost+=self.crowdedCost()
        return cost

    def visitBusStop(self,busStop,main):

        # fetch list of busStop objects
        busStops = main.getBusStops(self.route)

        for stop in busStops:
            if stop == busStop:
                # now the required BusStop object is stored in stop
                break
        
        # drop off code
        self.currentCapacity-=len(self.peopleDisembarkDict[busStop])
        self.peopleDisembarkDict[busStop] = []

        # implement pick up code here
        stop.pickUp(self)
    
    def crowdedCost(self):
        # if the number of people is above a threshold, start adding cost
        if self.currentCapacity > self.discomfortCapacity:
            return CROWDED_COST_MULTIPLIER * self.currentCapacity
        else:
            return 0
    
    def boardBus(self,curStop,person):
        disembarkAt = People.chooseDisembarkLocation(curStop,self.route)
        self.peopleDisembarkDict[disembarkAt].append(person)
        self.currentCapacity+=1

In [16]:
class BusStop:

    def __init__(self,name):
        
        # constants
        self.name = name
        self.maxCapacity = 60
        self.discomfortCapacity = 45

        # variables
        self.currentCapacity = 0
        self.people = []
        self.spawnRate = 0

    def __str__(self):
        return self.name + '   ' + str(self.currentCapacity)
        
    def __eq__(self,inp):
        if isinstance(inp,BusStop):
            if inp.name == self.name:
                return True
            else:
                return False

        elif isinstance(inp,str):
            if inp == self.name:
                return True
            else:
                return False

    def setSpawnRate(self,newSpawnRate):
        self.spawnRate = newSpawnRate

    def pickUp(self,bus):

        # store queue of people to board
        boardingPeople = self.people

        # function to make people board when it is crowded
        def crowdedBoard():
            # check queue of people
            # roll to check if they board or not
            while boardingPeople and bus.currentCapacity < bus.maxCapacity:
                # check if they can board by sampling a distribution
                if random.random() < (1-(bus.currentCapacity/bus.maxCapacity)):
                    # remove the person at the front of the queue
                    person = boardingPeople.pop(0)
                    
                    # change the status of the person
                    person.board()
                    # change the required variables in the bus
                    bus.boardBus(self.name,person)
                    # reduce the fullness of the bus stops
                    self.currentCapacity -= 1

        if bus.currentCapacity > bus.discomfortCapacity:
            crowdedBoard()
        else:
            if self.currentCapacity !=len(boardingPeople):
                print(self.currentCapacity,len(boardingPeople))
                raise ValueError
            for x in range(min(self.currentCapacity,bus.discomfortCapacity-bus.currentCapacity)):
                person = boardingPeople.pop(0)
                person.board()
                bus.boardBus(self.name,person)
                self.currentCapacity -= 1

            crowdedBoard()

    def spawn(self):
        # initialise cost
        cost = 0

        # generate the number of people to spawn and make sure that its greater than 0 = n
        numSpawn = numpy.random.normal(loc=self.spawnRate, scale=0.01, size=None)
        numSpawn = max(numSpawn,0)

        # try spawning n people
        for x in range(int(numSpawn)):

            # check if the current number of people in the bus stop is more than the threshold
            # if not keep spawning
            # if so roll to decide to spwan or not
            if self.currentCapacity<self.discomfortCapacity:
                # spwan a person
                person = People()
                self.people.append(person)
                self.currentCapacity+=1
            else:
                # if spwan is unsuccessful, increment cost
                if random.random() < (1-(self.currentCapacity/self.maxCapacity)):
                    # spwan a person
                    person = People()
                    self.people.append(person)
                    self.currentCapacity+=1
                else:
                    # increment cost
                    cost += REJECT_BUSSTOP_COST
        
        return cost

    def step(self):
        cost = 0
        cost += self.crowdedCost()
        cost += self.spawn()
        for person in self.people:
            cost+=person.step()
        return cost
        
    def crowdedCost(self):
        # if the number of people is above a threshold, start adding cost
        if self.currentCapacity > self.discomfortCapacity:
            return CROWDED_COST_MULTIPLIER * self.currentCapacity
        else:
            return 0

In [17]:
from Route import Route
class Main:

    def __init__(self):
        self.busArray = {"red":[],"blue":[]}
        self.busStops = {"red":[],"blue":[]} # initialise this
        redRouteDetails = Route.getDetails("red")
        blueRouteDetails = Route.getDetails("blue")
        
        for key in list(redRouteDetails.keys()):
            self.busStops['red'].append(BusStop(key))
            
        for key in list(blueRouteDetails.keys()):
            self.busStops['blue'].append(BusStop(key))
        
        self.timeOfDay = "Placeholder"

    def addBus(self,route):
        self.busArray[route].append(Bus(route))

    def scheduleRemoveBus(self,index):
        pass

    def __removeBus(self,route,index):
        busRemoved = self.busArray[route].pop(index)
        del busRemoved

    def getActiveBuses(self,route):
        return self.busArray[route]
    
    def step(self):
        cost = 0
        for bus in self.busArray['red']:
            cost += bus.step(self)
        
        for bus in self.busArray['blue']:
            cost += bus.step(self)
            
        for busStop in self.busStops['red']:
            cost += busStop.step()
        
        for busStop in self.busStops['blue']:
            cost += busStop.step()
        
        return cost
        
    def getBusStops(self,route):
        return self.busStops[route]

In [27]:
main = Main()

In [28]:
type(main)

__main__.Main

In [29]:
main.addBus('blue')

In [30]:
busses = main.getActiveBuses('blue')

In [31]:
busses

In [32]:
main.busStops['blue'][1].setSpawnRate(3)


In [33]:
cost = 0

In [34]:
# main.addBus('red')

In [35]:
for x in range(150):
    cost += main.step()
    print(main.busStops['blue'][1])
    print(main.busArray['blue'][0])
#     print(main.busArray['red'][1])
    print(cost)

 Opp Hall 3 & 16    2
0   (' Opp Hall 3 & 16 ', 0)
0.4
 Opp Hall 3 & 16    4
0   (' Opp Hall 3 & 16 ', 1)
1.2000000000000002
 Opp Hall 3 & 16    6
0   (' Opp Hall 14 & 15 ', 0)
2.4000000000000004
 Opp Hall 3 & 16    8
0   (' Opp Saraca Hall ', 0)
4.0
 Opp Hall 3 & 16    10
0   (' Opp Saraca Hall ', 1)
6.0
 Opp Hall 3 & 16    13
0   (' Opp Saraca Hall ', 2)
8.6
 Opp Hall 3 & 16    16
0   (' Opp Hall 11 ', 0)
11.8
 Opp Hall 3 & 16    19
0   (' Opp Hall 8 ', 0)
15.600000000000001
 Opp Hall 3 & 16    22
0   (' Opp Hall 8 ', 1)
20.000000000000004
 Opp Hall 3 & 16    25
0   (' Hall 6 ', 0)
25.000000000000007
 Opp Hall 3 & 16    28
0   (' Opp Hall 4 ', 0)
30.60000000000001
 Opp Hall 3 & 16    31
0   (' Opp Hall 4 ', 1)
36.80000000000001
 Opp Hall 3 & 16    34
0   (' Opp Yunnan Gardens ', 0)
43.600000000000016
 Opp Hall 3 & 16    37
0   (' Opp Yunnan Gardens ', 1)
51.00000000000002
 Opp Hall 3 & 16    39
0   (' Opp SPMS ', 0)
58.800000000000026
 Opp Hall 3 & 16    42
0   (' Opp SPMS ', 1)
67.2

[[1, 2, 3, 4], 0]